## Multiple source multiple sink Minimum Cost Flow (MCF) Problem

In [1]:
import numpy as np
import pandas as pd
import os
import igraph as ig
import networkx as nx
import random
from gurobipy import Model,GRB,LinExpr,quicksum
import cartopy
import cartopy.crs as ccrs
import matplotlib as mp
import matplotlib.pyplot as plt
import matplotlib.ticker as mticker
from matplotlib.axes._axes import _log as matplotlib_axes_logger
matplotlib_axes_logger.setLevel('ERROR')
from cartopy.mpl.gridliner import LONGITUDE_FORMATTER, LATITUDE_FORMATTER
import matplotlib.patheffects as PathEffects
import cartopy.io.shapereader as shpreader
import cartopy.feature as cfeature
import itertools
from pyomo.environ import *

V = {1:{'F':10},
     2:{'F':0},
     3:{'F':5},
     4:{'F':0},
     5:{'F':0},
     6:{'F':-15}}

E = {(1,2):{'U':8,'C':3},
     (1,3):{'U':6,'C':5},
     (2,3):{'U':5,'C':2},
     (2,4):{'U':5,'C':4},
     (3,2):{'U':7,'C':10},
     (3,4):{'U':10,'C':8},
     (3,5):{'U':4,'C':5},
     (4,5):{'U':8,'C':3},
     (4,6):{'U':5,'C':3},
     (5,6):{'U':10,'C':7}}

F =  {k:v['F'] for k,v in V.items()}
U =  {k:e['U'] for k,e in E.items()}
C =  {k:e['C'] for k,e in E.items()}

In [2]:
model = ConcreteModel()

# Define sets
model.vertices = Set(initialize=V.keys())
model.edges    = Set(initialize=E.keys())

# Define parameters
model.netflow  = Param(model.vertices, initialize=F, within=Reals)
model.capacity = Param(model.edges, initialize=U, within=NonNegativeReals)
model.cost     = Param(model.edges, initialize=C, within=NonNegativeReals)

# Define the decision variables
model.x = Var(model.edges, within=NonNegativeReals)

# Define the objective function
model.obj = Objective(expr=sum(model.cost[(u, v)] * model.x[(u, v)] for (u, v) in model.edges), sense=minimize)

# Define the constraints
model.flow_conservation = ConstraintList()
for v in model.vertices:
    model.flow_conservation.add(sum(model.x[(v, w)] for w in model.vertices if (v,w) in model.edges)-
                                sum(model.x[(w, v)] for w in model.vertices if (w,v) in model.edges)
                                == model.netflow[v])

model.upper_bound = ConstraintList()
for (v,w) in model.edges:
    model.upper_bound.add(model.x[(v, w)] <= model.capacity[(v,w)])
    
# Initialize the variables
for (v, w) in model.edges:
    model.x[(v, w)].value = 0

In [3]:
# Solve the problem
solver = SolverFactory('gurobi')
solver.solve(model)

# Print the results
print('Overall cost to process water:', model.obj())
for (v, w) in model.edges:
    print(f'Water flow along {v} - {w}: {model.x[(v, w)].value}')

Overall cost to process water: 231.0
Water flow along 1 - 2: 5.0
Water flow along 1 - 3: 5.0
Water flow along 2 - 3: 0.0
Water flow along 2 - 4: 5.0
Water flow along 3 - 2: 0.0
Water flow along 3 - 4: 6.0
Water flow along 3 - 5: 4.0
Water flow along 4 - 5: 6.0
Water flow along 4 - 6: 5.0
Water flow along 5 - 6: 10.0
